In [1]:
import json
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

In [2]:
#Opening our data of Institutions and Graduates per Institution.

In [3]:
Institutions = "resources/cc_institution_details.csv"
Institutions_db = pd.read_csv(Institutions,encoding="cp1252")

In [4]:
Grads = "resources/cc_institution_grads.csv"
Grads_db = pd.read_csv(Grads)

In [5]:
Institutions_db.head()

,unitid,chronname,city,state,level,control,basic,hbcu,flagship,long_x,...,vsa_grad_after6_transfer,vsa_grad_elsewhere_after6_transfer,vsa_enroll_after6_transfer,vsa_enroll_elsewhere_after6_transfer,similar,state_sector_ct,carnegie_ct,counted_pct,nicknames,cohort_size
0,100654,Alabama A&M University,Normal,Alabama,4-year,Public,Masters Colleges and Universities--larger prog...,X,NaN,-86.568502,...,36.4,5.6,17.2,11.1,232937|100724|405997|113607|139533|144005|2285...,13,386,99.7|07,NaN,882.0
1,100663,University of Alabama at Birmingham,Birmingham,Alabama,4-year,Public,Research Universities--very high research acti...,NaN,NaN,-86.809170,...,NaN,NaN,NaN,NaN,196060|180461|201885|145600|209542|236939|1268...,13,106,56.0|07,UAB,1376.0
2,100690,Amridge University,Montgomery,Alabama,4-year,Private not-for-profit,Baccalaureate Colleges--Arts & Sciences,NaN,NaN,-86.174010,...,NaN,NaN,NaN,NaN,217925|441511|205124|247825|197647|221856|1353...,16,252,100.0|07,NaN,3.0
3,100706,University of Alabama at Huntsville,Huntsville,Alabama,4-year,Public,Research Universities--very high research acti...,NaN,NaN,-86.638420,...,0.0,0.0,0.0,0.0,232186|133881|196103|196413|207388|171128|1900...,13,106,43.1|07,UAH,759.0
4,100724,Alabama State University,Montgomery,Alabama,4-year,Public,Masters Colleges and Universities--larger prog...,X,NaN,-86.295677,...,NaN,NaN,NaN,NaN,100654|232937|242617|243197|144005|241739|2354...,13,386,88.0|07,ASU,1351.0


In [6]:
#Grabbing the columns needed for our analysis

In [7]:
Inst = Institutions_db[["unitid","chronname","city","state","level","control","student_count","med_sat_value","aid_value","grad_100_value","grad_150_value"]]

In [8]:
Inst

,unitid,chronname,city,state,level,control,student_count,med_sat_value,aid_value,grad_100_value,grad_150_value
0,100654,Alabama A&M University,Normal,Alabama,4-year,Public,4051,823.0,7142.0,10.0,29.1
1,100663,University of Alabama at Birmingham,Birmingham,Alabama,4-year,Public,11502,1146.0,6088.0,29.4,53.5
2,100690,Amridge University,Montgomery,Alabama,4-year,Private not-for-profit,322,NaN,2540.0,0.0,66.7
3,100706,University of Alabama at Huntsville,Huntsville,Alabama,4-year,Public,5696,1180.0,6647.0,16.5,48.4
4,100724,Alabama State University,Montgomery,Alabama,4-year,Public,5356,830.0,7256.0,8.8,25.2
...,...,...,...,...,...,...,...,...,...,...,...
3793,461528,Grace College of Divinity,Fayetteville,North Carolina,4-year,Private not-for-profit,110,NaN,4860.0,0.0,0.0
3794,462354,John Paul the Great Catholic University,Escondido,California,4-year,Private not-for-profit,140,1069.0,9395.0,51.5,51.5
3795,466921,Chamberlain College of Nursing-Missouri,St. Louis,Missouri,4-year,Private for-profit,488,NaN,3291.0,37.5,37.5
3796,474881,Minneapolis Media Institute,Edina,Minnesota,2-year,Private for-profit,147,NaN,3434.0,42.1,55.9


In [9]:
#Filtering colleges for 4 years only

In [10]:
Inst2 = Inst.loc[(Inst["level"] == "4-year")]

In [11]:
len(Inst2)

2339

In [12]:
#Dropping all null values

In [13]:
Inst2 = Inst2.dropna()

In [14]:
len(Inst2)

1297

In [15]:
Howmanystates = Inst2.groupby(["state"]).count()

In [16]:
#Counting States of study

In [17]:
len(Howmanystates)

51

In [18]:
Grads_db

,unitid,year,gender,race,cohort,grad_cohort,grad_100,grad_150,grad_100_rate,grad_150_rate
0,100760,2011,B,X,2y all,446.0,73.0,105.0,16.4,23.5
1,100760,2011,M,X,2y all,185.0,NaN,40.0,NaN,21.6
2,100760,2011,F,X,2y all,261.0,NaN,65.0,NaN,24.9
3,100760,2011,B,W,2y all,348.0,NaN,86.0,NaN,24.7
4,100760,2011,M,W,2y all,162.0,NaN,35.0,NaN,21.6
...,...,...,...,...,...,...,...,...,...,...
1302097,168591,2002,F,Ai,4y other,NaN,NaN,NaN,NaN,NaN
1302098,168740,2002,F,Ai,4y other,0.0,NaN,0.0,NaN,NaN
1302099,169716,2002,F,Ai,4y other,NaN,NaN,NaN,NaN,NaN
1302100,170082,2002,F,Ai,4y other,NaN,NaN,NaN,NaN,NaN


In [19]:
len(Grads_db)

1302102

In [20]:
#We dropped a lot of records from the Institution database. Now, we have to get rid of all 
#the Institutions we have in Grads data set to have the same amount of schools 
#in the institutions db and the Grads_db to maintain data integrity.

In [21]:
b=Inst2["unitid"].unique()
b2=Grads_db["unitid"].unique()


In [22]:
len(b)

1297

In [23]:
len(b2)
#As seen here b and b2 don't have the same number of institutions
#in Grads data set

3979

In [24]:
#This following code creates an array with the schools that are not in InstitutionDB but exsists in Grads_db so we 
#can drop those students without schools, we expect a good amount of schools to be dropped from Grads.

In [25]:
StudentswithoutSchools=[]
for index, row in Grads_db.iterrows():
   
    School = row["unitid"]
   # print(School)
    SchoolsinInst = Inst2.loc[(Inst2["unitid"] == School)] 
    InstFlag = SchoolsinInst.empty
    if InstFlag == True:
        if School not in StudentswithoutSchools:
            StudentswithoutSchools.append(School)

In [27]:
len(StudentswithoutSchools)


2682

In [28]:
#The following code creates an array with the schools that are in Institution but not in Grads_db(Just a test) 
#because we expect to get zero, because we only dropped data from Institutions data base.
#So this test is just to make sure they had data integrity before we start using these tables for our analysis

In [29]:
SchoolsWOStudents=[]
for index, row in Inst2.iterrows():
   
    School = row["unitid"]
   # print(School)
    SchoolsinInst = Grads_db.loc[(Grads_db["unitid"] == School)] 
    InstFlag = SchoolsinInst.empty
    if InstFlag == True:
        if School not in SchoolsWOStudents:
            SchoolsWOStudents.append(School)   

In [30]:
#This is expected to be zero because we didn't get rid of anything from Grads db yet
len(SchoolsWOStudents)

0

In [31]:
#Crating a dataframe from the array of schools to drop from Grads data set

In [32]:
columns = ["school"]
Table = pd.DataFrame(data=StudentswithoutSchools,columns=columns)

In [33]:
len(Table)

2682

In [34]:
len(Grads_db)

1302102

In [35]:
#This will drop all the students that have no school in the main Institution data.---It dropped 2682 schools that we had null values
#that made 744390 records dropped out of 1302102 from the Grads data
for index, row in Table.iterrows():
       
    School = row["school"]
    print(School)
    indexnumbers = Grads_db[Grads_db['unitid']==School].index
    Grads_db.drop(indexnumbers,inplace=True)

100760
101028
101143
101161
101240
101286
101295
101301
101462
101499
101505
101514
101569
101602
101648
101736
101897
101949
101958
101994
102030
102067
102076
102313
102429
103361
103909
103927
104160
104346
104425
104577
104708
105118
105145
105154
105206
105349
105428
105525
105543
105659
105668
105747
105792
106041
106148
106449
106625
106795
106883
106980
106999
107318
107327
107460
107521
107549
107585
107619
107637
107664
107725
107743
107974
107992
108667
108807
108852
109040
109208
109350
109819
109907
110246
110334
111009
111434
111461
111638
111887
111896
111920
111939
112172
112190
112385
112561
112686
112826
113096
113111
113193
113218
113236
113333
113573
113634
113856
113980
114123
114266
114415
114433
114716
114789
114859
114938
115001
115126
115296
115393
115454
115472
115490
115515
115533
115861
116439
117052
117195
117247
117274
117645
117690
117706
117715
117724
117733
117788
117867
117894
118198
118259
118277
118286
118347
118684
118718
118772
118912
118930
118976

375841
375939
377111
377342
377449
377458
379029
380094
380359
380368
380438
381732
382504
382771
382896
382911
382957
383190
383950
383996
384209
384333
384342
385008
385503
387925
388043
389727
392257
395362
398574
399212
403469
403478
403487
404055
404365
404374
404383
404426
404499
404505
404514
404912
404921
404994
405872
406024
406033
407328
408394
409670
409874
410070
410283
410496
410520
410539
413626
414841
414911
417327
417734
418870
418889
420024
420103
420398
420404
420431
420440
420495
420501
420538
420556
420723
421513
421708
421878
422020
423652
423980
427973
427982
428170
428329
428392
429128
430050
430148
430157
430166
430351
430360
430795
431141
431886
432302
432524
433013
433174
433563
434016
434061
434140
434159
434566
434575
434584
434672
434690
434751
434821
436182
436225
436304
437060
437103
437732
437750
437769
437936
438212
438258
438285
438294
438647
438708
438869
439118
439127
439145
439190
439367
439437
439455
439482
439525
439570
439738
439792
439826
440110

445133
445258
445276
445300
445319
445391
445434
445708
445717
445920
445939
446233
446604
446640
446668
446677
446738
446747
446756
446765
446792
446808
446899
446905
446914
446923
447050
447069
447263
447272
447290
447360
447421
447458
447670
447689
447953
448309
448345
448442
448451
448479
448488
448497
448503
448512
448576
448585
448628
448673
448734
448761
449010
449038
449658
449870
449898
450049
450085
450094
450207
450216
450225
450234
450243
450252
450261
450270
450298
450377
450429
450447
450465
450526
450535
450571
450933
451060
451404
451556
451662
451671
451699
451750
451769
451796
451820
451848
451875
451936
451945
451954
451963
451972
451981
451990
452018
452027
453127
453163
453215
454227
454236
454245
454591
454856
454883
455257
455585
455600
455619
456010
456056
456296
456409
456427
456445
456490
456612
456667
456782
456791
456807
456816
456834
457022
457484
457688
458210
458229
458256
458432
458502
458557
458733
458742
458760
458779
458919
458964
458973
458991
459204

In [36]:
#Ending with this ammount of students

In [37]:
len(Grads_db)

557712

In [38]:
#found out we had to get rid of the distribution for gender and ethnicity by just grabing both genders
#and all races all sumarized.


In [39]:
BothGenders = Grads_db.loc[(Grads_db["gender"] == "B") & (Grads_db["race"] == "X")]

In [40]:
#This was the last step of the cleanng of our data, to end up with a Grads data set by year and by Institution, 
#and Institution data set by state. Keeping data ingerity among the two data sets and their institutions.



In [41]:
Grads_db2 = BothGenders.groupby(["unitid","year"], as_index=False).sum()

In [42]:
Grads_db2

,unitid,year,grad_cohort,grad_100,grad_150,grad_100_rate,grad_150_rate
0,100654,2002,716.0,147.0,278.0,20.5,38.8
1,100654,2003,789.0,159.0,300.0,20.2,38.0
2,100654,2004,995.0,182.0,326.0,18.3,32.8
3,100654,2005,1111.0,97.0,355.0,8.7,32.0
4,100654,2006,1020.0,118.0,342.0,11.6,33.5
...,...,...,...,...,...,...,...
15487,454184,2012,78.0,41.0,45.0,52.6,57.7
15488,454184,2013,44.0,25.0,27.0,56.8,61.4
15489,462354,2011,0.0,0.0,0.0,0.0,0.0
15490,462354,2012,19.0,12.0,12.0,63.2,63.2


In [43]:
Grads_db2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15492 entries, 0 to 15491
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   unitid         15492 non-null  int64  
 1   year           15492 non-null  int64  
 2   grad_cohort    15492 non-null  float64
 3   grad_100       15492 non-null  float64
 4   grad_150       15492 non-null  float64
 5   grad_100_rate  15492 non-null  float64
 6   grad_150_rate  15492 non-null  float64
dtypes: float64(5), int64(2)
memory usage: 968.2 KB


In [44]:
#These datasets were saved to be used in our data analisys

In [45]:
Grads_db2.to_csv('resources/Grads_db2.csv')

In [46]:
Inst2.to_csv('resources/Inst2.csv')

In [47]:
#The following code is to find our max number of Graduates per each state.

In [48]:
#Opening the dataset for the population from 2002 to 2009

In [49]:
Population9 = "resources/population2009.csv"
population09 = pd.read_csv(Population9)
#open csv

In [50]:
population09.head()

,State,1-Jul-09,1-Jul-08,1-Jul-07,1-Jul-06,1-Jul-05,1-Jul-04,1-Jul-03,1-Jul-02
0,Alabama,"4,708,708","4,677,464","4,637,904","4,597,688","4,545,049","4,512,190","4,490,591","4,472,420"
1,Alaska,"698,473","688,125","682,297","677,325","669,488","661,569","650,884","642,691"
2,Arizona,"6,595,778","6,499,377","6,362,241","6,192,100","5,974,834","5,759,425","5,591,206","5,452,108"
3,Arkansas,"2,889,450","2,867,764","2,842,194","2,815,097","2,776,221","2,746,161","2,722,291","2,704,732"
4,California,"36,961,664","36,580,371","36,226,122","35,979,208","35,795,255","35,558,419","35,251,107","34,876,194"


In [51]:
#Opening the dataset for census from 2010 to 2013

In [52]:
Population13 = "resources/poppulation2013.csv"
population013 = pd.read_csv(Population13,encoding="cp1252")

In [53]:
pop13 = population013[["State","2010","2011","2012","2013"]]

In [54]:
pop13.head()

,State,2010,2011,2012,2013
0,Alabama,"4,785,161","4,801,108","4,816,089","4,830,533"
1,Alaska,"714,021","722,720","731,228","737,442"
2,Arizona,"6,408,208","6,468,732","6,553,262","6,630,799"
3,Arkansas,"2,922,394","2,938,538","2,949,499","2,957,957"
4,California,"37,334,079","37,700,034","38,056,055","38,414,128"


In [55]:
dict = {"1-Jul-09": "2009","1-Jul-08":"2008","1-Jul-07":"2007","1-Jul-06":"2006","1-Jul-05":"2005","1-Jul-04":"2004","1-Jul-03":"2003","1-Jul-02":"2002"}
pop09 = population09.rename(columns = dict)


In [56]:
pop09.head()

,State,2009,2008,2007,2006,2005,2004,2003,2002
0,Alabama,"4,708,708","4,677,464","4,637,904","4,597,688","4,545,049","4,512,190","4,490,591","4,472,420"
1,Alaska,"698,473","688,125","682,297","677,325","669,488","661,569","650,884","642,691"
2,Arizona,"6,595,778","6,499,377","6,362,241","6,192,100","5,974,834","5,759,425","5,591,206","5,452,108"
3,Arkansas,"2,889,450","2,867,764","2,842,194","2,815,097","2,776,221","2,746,161","2,722,291","2,704,732"
4,California,"36,961,664","36,580,371","36,226,122","35,979,208","35,795,255","35,558,419","35,251,107","34,876,194"


In [57]:
Population = pd.merge(pop09,pop13, how="inner", on="State")

In [58]:
Population.head()

,State,2009,2008,2007,2006,2005,2004,2003,2002,2010,2011,2012,2013
0,Alabama,"4,708,708","4,677,464","4,637,904","4,597,688","4,545,049","4,512,190","4,490,591","4,472,420","4,785,161","4,801,108","4,816,089","4,830,533"
1,Alaska,"698,473","688,125","682,297","677,325","669,488","661,569","650,884","642,691","714,021","722,720","731,228","737,442"
2,Arizona,"6,595,778","6,499,377","6,362,241","6,192,100","5,974,834","5,759,425","5,591,206","5,452,108","6,408,208","6,468,732","6,553,262","6,630,799"
3,Arkansas,"2,889,450","2,867,764","2,842,194","2,815,097","2,776,221","2,746,161","2,722,291","2,704,732","2,922,394","2,938,538","2,949,499","2,957,957"
4,California,"36,961,664","36,580,371","36,226,122","35,979,208","35,795,255","35,558,419","35,251,107","34,876,194","37,334,079","37,700,034","38,056,055","38,414,128"


In [59]:
#After we merged the two datasets of estimated census per year we have now the table with census complete from 2002 to 2013

In [60]:
#The following command is to convert the columns into rows so we can merge with our grads table by state

In [61]:
Pop =Population.melt(id_vars=["State"], 
        var_name="Date", 
        value_name="Population")

In [62]:
Pop

,State,Date,Population
0,Alabama,2009,"4,708,708"
1,Alaska,2009,"698,473"
2,Arizona,2009,"6,595,778"
3,Arkansas,2009,"2,889,450"
4,California,2009,"36,961,664"
...,...,...,...
619,Washington,2013,"6,973,281"
620,West Virginia,2013,"1,852,985"
621,Wisconsin,2013,"5,743,653"
622,Wyoming,2013,"583,131"


In [63]:
Grads = Grads_db2

In [64]:
Schools = Inst2

In [65]:
len(Grads)

15492

In [66]:
Grads.head()

,unitid,year,grad_cohort,grad_100,grad_150,grad_100_rate,grad_150_rate
0,100654,2002,716.0,147.0,278.0,20.5,38.8
1,100654,2003,789.0,159.0,300.0,20.2,38.0
2,100654,2004,995.0,182.0,326.0,18.3,32.8
3,100654,2005,1111.0,97.0,355.0,8.7,32.0
4,100654,2006,1020.0,118.0,342.0,11.6,33.5


In [67]:
Grads2 = pd.merge(Grads,Schools, how="outer", on="unitid")

In [68]:
len(Grads2)

15492

In [69]:
Grads2.head()

,unitid,year,grad_cohort,grad_100,grad_150,grad_100_rate,grad_150_rate,chronname,city,state,level,control,student_count,med_sat_value,aid_value,grad_100_value,grad_150_value
0,100654,2002,716.0,147.0,278.0,20.5,38.8,Alabama A&M University,Normal,Alabama,4-year,Public,4051,823.0,7142.0,10.0,29.1
1,100654,2003,789.0,159.0,300.0,20.2,38.0,Alabama A&M University,Normal,Alabama,4-year,Public,4051,823.0,7142.0,10.0,29.1
2,100654,2004,995.0,182.0,326.0,18.3,32.8,Alabama A&M University,Normal,Alabama,4-year,Public,4051,823.0,7142.0,10.0,29.1
3,100654,2005,1111.0,97.0,355.0,8.7,32.0,Alabama A&M University,Normal,Alabama,4-year,Public,4051,823.0,7142.0,10.0,29.1
4,100654,2006,1020.0,118.0,342.0,11.6,33.5,Alabama A&M University,Normal,Alabama,4-year,Public,4051,823.0,7142.0,10.0,29.1


In [70]:
#We extract just the info we will need for the grads data set. We needed to get the state of the school in our Grads table.

In [71]:
Grads = Grads2[["unitid","state","year","control","grad_cohort","grad_100","grad_150"]]

In [72]:
Grads

,unitid,state,year,control,grad_cohort,grad_100,grad_150
0,100654,Alabama,2002,Public,716.0,147.0,278.0
1,100654,Alabama,2003,Public,789.0,159.0,300.0
2,100654,Alabama,2004,Public,995.0,182.0,326.0
3,100654,Alabama,2005,Public,1111.0,97.0,355.0
4,100654,Alabama,2006,Public,1020.0,118.0,342.0
...,...,...,...,...,...,...,...
15487,454184,New York,2012,Private not-for-profit,78.0,41.0,45.0
15488,454184,New York,2013,Private not-for-profit,44.0,25.0,27.0
15489,462354,California,2011,Private not-for-profit,0.0,0.0,0.0
15490,462354,California,2012,Private not-for-profit,19.0,12.0,12.0


In [73]:
ByStateYear = Grads.groupby(["state","year"],as_index=False).sum()

In [74]:
ByStateYear

,state,year,unitid,grad_cohort,grad_100,grad_150
0,Alabama,2002,2232644,15667.0,4187.0,7723.0
1,Alabama,2003,2232644,15988.0,4123.0,7916.0
2,Alabama,2004,2232644,16960.0,4183.0,8146.0
3,Alabama,2005,2232644,17758.0,4402.0,8551.0
4,Alabama,2006,2232644,17921.0,4599.0,8698.0
...,...,...,...,...,...,...
607,Wyoming,2009,240727,1388.0,310.0,767.0
608,Wyoming,2010,240727,1496.0,336.0,793.0
609,Wyoming,2011,240727,1399.0,320.0,744.0
610,Wyoming,2012,240727,1543.0,359.0,840.0


In [75]:
Pop["Date"]= Pop["Date"].astype(int)

In [76]:
ByStateYearplusPop = pd.merge(ByStateYear,Pop, how="inner", left_on =["state","year"], right_on=["State","Date"])

In [77]:
#Finally here we have the Grads data set with the state and also the population of the state so we can calculate
#the graduation rate in relation of the population of the state per year.

In [78]:
ByStateYearplusPop

,state,year,unitid,grad_cohort,grad_100,grad_150,State,Date,Population
0,Alabama,2002,2232644,15667.0,4187.0,7723.0,Alabama,2002,"4,472,420"
1,Alabama,2003,2232644,15988.0,4123.0,7916.0,Alabama,2003,"4,490,591"
2,Alabama,2004,2232644,16960.0,4183.0,8146.0,Alabama,2004,"4,512,190"
3,Alabama,2005,2232644,17758.0,4402.0,8551.0,Alabama,2005,"4,545,049"
4,Alabama,2006,2232644,17921.0,4599.0,8698.0,Alabama,2006,"4,597,688"
...,...,...,...,...,...,...,...,...,...
607,Wyoming,2009,240727,1388.0,310.0,767.0,Wyoming,2009,"544,270"
608,Wyoming,2010,240727,1496.0,336.0,793.0,Wyoming,2010,"564,516"
609,Wyoming,2011,240727,1399.0,320.0,744.0,Wyoming,2011,"567,768"
610,Wyoming,2012,240727,1543.0,359.0,840.0,Wyoming,2012,"577,080"


In [79]:
#New column for the total of Grads per state per year including both within 4 and within 6 years

In [80]:
ByStateYearplusPop["Total100pls150"] = ByStateYearplusPop["grad_100"] + ByStateYearplusPop["grad_150"]

In [81]:
ByStateYearplusPop = ByStateYearplusPop[["state","year","grad_100","grad_150","Population","Total100pls150"]]

In [82]:
ByStateYearplusPop

,state,year,grad_100,grad_150,Population,Total100pls150
0,Alabama,2002,4187.0,7723.0,"4,472,420",11910.0
1,Alabama,2003,4123.0,7916.0,"4,490,591",12039.0
2,Alabama,2004,4183.0,8146.0,"4,512,190",12329.0
3,Alabama,2005,4402.0,8551.0,"4,545,049",12953.0
4,Alabama,2006,4599.0,8698.0,"4,597,688",13297.0
...,...,...,...,...,...,...
607,Wyoming,2009,310.0,767.0,"544,270",1077.0
608,Wyoming,2010,336.0,793.0,"564,516",1129.0
609,Wyoming,2011,320.0,744.0,"567,768",1064.0
610,Wyoming,2012,359.0,840.0,"577,080",1199.0


In [83]:
#replace the commas with spaces to make it into a numeric fiels for our per capita calculations

In [84]:
ByStateYearplusPop= ByStateYearplusPop.replace(to_replace=',',value='', regex = True)

In [85]:
ByStateYearplusPop["Population"]= ByStateYearplusPop["Population"].astype(float)

In [86]:
ByStateYearplusPop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 612 entries, 0 to 611
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   state           612 non-null    object 
 1   year            612 non-null    int64  
 2   grad_100        612 non-null    float64
 3   grad_150        612 non-null    float64
 4   Population      612 non-null    float64
 5   Total100pls150  612 non-null    float64
dtypes: float64(4), int64(1), object(1)
memory usage: 33.5+ KB


In [87]:
ByStateYearplusPop

,state,year,grad_100,grad_150,Population,Total100pls150
0,Alabama,2002,4187.0,7723.0,4472420.0,11910.0
1,Alabama,2003,4123.0,7916.0,4490591.0,12039.0
2,Alabama,2004,4183.0,8146.0,4512190.0,12329.0
3,Alabama,2005,4402.0,8551.0,4545049.0,12953.0
4,Alabama,2006,4599.0,8698.0,4597688.0,13297.0
...,...,...,...,...,...,...
607,Wyoming,2009,310.0,767.0,544270.0,1077.0
608,Wyoming,2010,336.0,793.0,564516.0,1129.0
609,Wyoming,2011,320.0,744.0,567768.0,1064.0
610,Wyoming,2012,359.0,840.0,577080.0,1199.0


In [88]:
ByStateYearplusPop["TotalPercapita"] = (ByStateYearplusPop["Total100pls150"]/(ByStateYearplusPop["Population"]))*100000

In [89]:
ByStateYearplusPop

,state,year,grad_100,grad_150,Population,Total100pls150,TotalPercapita
0,Alabama,2002,4187.0,7723.0,4472420.0,11910.0,266.298782
1,Alabama,2003,4123.0,7916.0,4490591.0,12039.0,268.093888
2,Alabama,2004,4183.0,8146.0,4512190.0,12329.0,273.237607
3,Alabama,2005,4402.0,8551.0,4545049.0,12953.0,284.991427
4,Alabama,2006,4599.0,8698.0,4597688.0,13297.0,289.210577
...,...,...,...,...,...,...,...
607,Wyoming,2009,310.0,767.0,544270.0,1077.0,197.879729
608,Wyoming,2010,336.0,793.0,564516.0,1129.0,199.994331
609,Wyoming,2011,320.0,744.0,567768.0,1064.0,187.400488
610,Wyoming,2012,359.0,840.0,577080.0,1199.0,207.770153


In [90]:
#Now that we have the total of Grads per capita we can single out per year which state had the most grads using the total grads
#percapita
#The following code is to single out the state with the max number of grads percapita.

In [91]:
inx = ByStateYearplusPop.groupby(['year'])["TotalPercapita"].transform(max)==ByStateYearplusPop["TotalPercapita"]

In [92]:
ByStateYearplusPop[inx]

,state,year,grad_100,grad_150,Population,Total100pls150,TotalPercapita
96,District of Columbia,2002,3718.0,4220.0,579585.0,7938.0,1369.600663
97,District of Columbia,2003,3859.0,4515.0,577777.0,8374.0,1449.348105
98,District of Columbia,2004,4253.0,4902.0,579796.0,9155.0,1579.003650
99,District of Columbia,2005,4413.0,5075.0,582049.0,9488.0,1630.103307
100,District of Columbia,2006,4604.0,5281.0,583978.0,9885.0,1692.700752
101,District of Columbia,2007,5224.0,5843.0,586409.0,11067.0,1887.249343
102,District of Columbia,2008,5047.0,5658.0,590074.0,10705.0,1814.179239
103,District of Columbia,2009,5056.0,5645.0,599657.0,10701.0,1784.520151
104,District of Columbia,2010,5351.0,6052.0,605126.0,11403.0,1884.400935
105,District of Columbia,2011,5244.0,5863.0,620472.0,11107.0,1790.088836


In [93]:
#Our findings were to expect since we have a good amout of students graduating from DC and it si a very small population


In [94]:
#Then we found the sate with the Max number of grads per year without considering the population per state.
#The following code is to single out the state with the max number of grads.

In [95]:
inx2 = ByStateYearplusPop.groupby(['year'])["Total100pls150"].transform(max)==ByStateYearplusPop["Total100pls150"]

In [96]:
ByStateYearplusPop[inx2]

,state,year,grad_100,grad_150,Population,Total100pls150,TotalPercapita
52,California,2006,30130.0,53578.0,35979208.0,83708.0,232.656594
53,California,2007,32254.0,56047.0,36226122.0,88301.0,243.749524
54,California,2008,35074.0,59181.0,36580371.0,94255.0,257.665511
55,California,2009,36257.0,60826.0,36961664.0,97083.0,262.658629
56,California,2010,37111.0,60521.0,37334079.0,97632.0,261.509063
57,California,2011,38549.0,63822.0,37700034.0,102371.0,271.540869
58,California,2012,41881.0,69958.0,38056055.0,111839.0,293.879647
59,California,2013,42673.0,71849.0,38414128.0,114522.0,298.124690
384,New York,2002,27845.0,41294.0,19161873.0,69139.0,360.815459
385,New York,2003,30443.0,44596.0,19231101.0,75039.0,390.196068


In [97]:
#We found CALIFORNIA had the most grads in 2013 and it was very consistent througout the most recent years.
#So we decided to go with this state due to the ammount of schools that were involved in this study.
#Now that we have a year and state with the most graduates we are going to analyze the relationships to why this state has the most grads.



In [98]:
California = Grads.loc[(Grads["state"]=="California") & (Grads["year"]==2013)]

In [99]:
California

,unitid,state,year,control,grad_cohort,grad_100,grad_150
563,109785,California,2013,Private not-for-profit,848.0,399.0,550.0
575,110097,California,2013,Private not-for-profit,789.0,385.0,538.0
587,110361,California,2013,Private not-for-profit,489.0,200.0,284.0
599,110404,California,2013,Private not-for-profit,231.0,190.0,215.0
611,110413,California,2013,Private not-for-profit,424.0,260.0,297.0
...,...,...,...,...,...,...,...
15374,399911,California,2013,Private not-for-profit,101.0,79.0,85.0
15386,409698,California,2013,Public,734.0,88.0,281.0
15410,441937,California,2013,Public,512.0,106.0,264.0
15437,445188,California,2013,Public,668.0,179.0,379.0


In [100]:
California.to_csv("resources/California.csv")